In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns
sns.set(style="whitegrid")

In [3]:
original_df=pd.read_excel('附件1/附件1.xlsx',sheet_name='Data')

In [143]:
original_df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,199801010001,1998,1,1,NaN,0,NaT,34,Burundi,11,...,NaN,"“Burundi Rebels, Ex-Rwandan Army Soldiers Blam...",“Burundi--Attack Reported on Bujumbura Airport...,NaN,CETIS,0,1,0,1,NaN
1,199801010002,1998,1,1,NaN,0,NaT,167,Russia,9,...,NaN,"“Bomb injures 3 in Moscow subway system,” The ...","“Bomb injures 3 in Moscow subway,” Charleston ...","“Bomb Injures 3 Workers in Moscow Metro,” Los ...",CETIS,-9,-9,0,-9,NaN
2,199801010003,1998,1,1,NaN,0,NaT,603,United Kingdom,8,...,NaN,“Protestant gunmen kill Catholic in New Year's...,“Ulster Peace Shattered by Shooting: Catholic ...,NaN,CETIS,0,0,1,1,NaN
3,199801020001,1998,1,2,NaN,0,NaT,95,Iraq,10,...,NaN,“Iraq Condemns Attack on UNSCOM Baghdad Office...,"Farouk Choukri , “Iraq, UN Officials Continue ...","“Iraqi Interior Minister on UNSCOM Attack, Kuw...",CETIS,-9,-9,1,1,NaN
4,199801020002,1998,1,2,NaN,0,NaT,155,West Bank and Gaza Strip,10,...,NaN,"“Woman Shot,” The Philadelphia Inquirer, Janua...",“Israeli Woman Critically Hurt by Gunfire in W...,NaN,CETIS,-9,-9,0,-9,NaN


In [144]:
df=original_df.copy()

In [157]:
df["y-m"]=df["iyear"].map(str)+'-'+((df["imonth"]-1)//3+1).map(str)

In [158]:
df["y-m"]

0        2015-1
1        2015-1
2        2015-1
3        2015-1
4        2015-1
          ...  
39447    2017-4
39448    2017-4
39449    2017-4
39450    2017-4
39451    2017-4
Name: y-m, Length: 39452, dtype: object

In [186]:
df_ym_num=df.groupby("y-m").size()

In [378]:
df_ym_num

y-m
2015-1    4012
2015-2    3761
2015-3    3660
2015-4    3532
2016-1    3460
2016-2    3629
2016-3    3321
2016-4    3177
2017-1    2719
2017-2    3023
2017-3    2800
2017-4    2358
dtype: int64

In [385]:
df_ym_num.to_csv('ym_num.csv')

In [386]:
class GrayForecast():
    # 初始化
    def __init__(self, data, n):
        """
        :param data: Series/np/list
        :param n: 预测数量
        """
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        if isinstance(data, pd.Series):
            self.data = data.values
        elif isinstance(data, np.ndarray):
            self.data = data
        elif isinstance(data, list):
            self.data = np.array(data)
        self.level_check()
        self.GM_11_build_model(n)
        print("返回值为dataframe，可通过.res_df拿到, 可通过.plot_res画预测图\n", self.res_df)

    def level_check(self):
        # 数据级比校验
        b = self.data[0]
        n = len(self.data)
        lambda_k = np.zeros(n - 1)
        while (True):
            for i in range(n - 1):
                lambda_k[i] = self.data[i] / self.data[i + 1]
            if max(lambda_k) < np.exp(2 / (n + 2)) and min(lambda_k) > np.exp(-2 / (n + 1)):
                self.c = self.data[0] - b
                print(f"完成数据 级比校验, 平移变换c={self.c}")
                break
            else:
                self.data = self.data + 0.1

    # GM(1,1)建模
    def GM_11_build_model(self, n):
        '''
            灰色预测
            x：序列，numpy对象
            n:需要往后预测的个数
        '''
        x = self.data
        # 累加生成（1-AGO）序列
        x1 = x.cumsum()
        # 紧邻均值生成序列
        z1 = (x1[:len(x1) - 1] + x1[1:]) / 2.0
        z1 = z1.reshape((len(z1), 1))
        B = np.append(-z1, np.ones_like(z1), axis=1)
        Y = x[1:].reshape((len(x) - 1, 1))
        # a为发展系数 b为灰色作用量
        [[a], [b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)  # 计算参数
        # 预测数据
        fit_res = [x[0]]
        for index in range(1, len(x) + n):
            fit_res.append((x[0] - b / a) * (1 - np.exp(a)) * np.exp(-a * (index)))
        # 数据还原
        self.data -= self.c
        fit_res -= self.c
        self.res_df = pd.concat([pd.DataFrame({'原始值': self.data}), pd.DataFrame({'预测值': fit_res})], axis=1)
        print(f"发展系数a={a}, 灰色作用量b={b}\n")
        self.verfify(self.data, fit_res, a)
        return self.res_df

    def verfify(self, x, predict, a):
        S1_2 = x.var()  # 原序列方差
        e = list()  # 残差序列
        for index in range(x.shape[0]):
            e.append(x[index] - predict[index])
        S2_2 = np.array(e).var()  # 残差方差
        C = S2_2 / S1_2  # 后验差比
        if C <= 0.35:
            assess = '后验差比<=0.35，模型精度等级为好'
        elif C <= 0.5:
            assess = '后验差比<=0.5，模型精度等级为合格'
        elif C <= 0.65:
            assess = '后验差比<=0.65，模型精度等级为勉强'
        else:
            assess = '后验差比>0.65，模型精度等级为不合格'
        print(f"后验差比={C}, {assess} \n")

        # 级比偏差
        a_ = (1 - 0.5 * a) / (1 + 0.5 * a)
        delta = [np.nan]
        for i in range(x.shape[0] - 1):
            delta.append(1 - a_ * (x[i] / x[i + 1]))

        self.res_df = pd.concat([self.res_df, pd.DataFrame({'残差': e}),
                                 pd.DataFrame({'相对误差': list(map(lambda x: '{:.2%}'.format(x), np.abs(e / x)))}),
                                 pd.DataFrame({'级比偏差': delta})
                                 ],
                                axis=1)

    def plot_res(self, xlabel='', ylabel=''):
        res_df = self.res_df
        f, ax = plt.subplots(figsize=(8, 5))
        sns.lineplot(x=res_df.index.tolist(), y=res_df['预测值'], linewidth=2, ax=ax)
        sns.scatterplot(x=res_df.index.tolist(), y=res_df['原始值'], s=60, color='r', marker='v', ax=ax)
        plt.fill_between(np.where(np.isnan(res_df["原始值"]))[0], y1=min(plt.yticks()[0]), y2=max(plt.yticks()[0]),
                         color='orange', alpha=0.2)
        ax.set_xlabel(xlabel, fontsize=15)
        ax.set_ylabel(ylabel, fontsize=15)
        plt.show()


In [387]:
gm_model=GrayForecast(df_ym_num/1000,10)

完成数据 级比校验, 平移变换c=0.5999999999999979
发展系数a=0.03244780698095551, 灰色作用量b=4.6945697959855766

后验差比=0.11672033342303606, 后验差比<=0.35，模型精度等级为好 

返回值为dataframe，可通过.res_df拿到, 可通过.plot_res画预测图
       原始值       预测值        残差    相对误差      级比偏差
0   4.012  4.012000  0.000000   0.00%       NaN
1   3.761  3.871975 -0.110975   2.95% -0.032677
2   3.660  3.729198 -0.069198   1.89%  0.005215
3   3.532  3.590980 -0.058980   1.67% -0.003153
4   3.460  3.457174  0.002826   0.08%  0.011785
5   3.629  3.327641  0.301359   8.30%  0.077012
6   3.321  3.202243  0.118757   3.58% -0.057852
7   3.177  3.080849  0.096151   3.03% -0.011949
8   2.719  2.963330 -0.244330   8.99% -0.131136
9   3.023  2.849564  0.173436   5.74%  0.129281
10  2.800  2.739429  0.060571   2.16% -0.045170
11  2.358  2.632811 -0.274811  11.65% -0.149532
12    NaN  2.529597       NaN     NaN       NaN
13    NaN  2.429679       NaN     NaN       NaN
14    NaN  2.332950       NaN     NaN       NaN
15    NaN  2.239310       NaN     NaN       NaN


In [388]:
res_df=gm_model.res_df

In [389]:
res_df

,原始值,预测值,残差,相对误差,级比偏差
0,4.012,4.012000,0.000000,0.00%,NaN
1,3.761,3.871975,-0.110975,2.95%,-0.032677
2,3.660,3.729198,-0.069198,1.89%,0.005215
3,3.532,3.590980,-0.058980,1.67%,-0.003153
4,3.460,3.457174,0.002826,0.08%,0.011785
5,3.629,3.327641,0.301359,8.30%,0.077012
6,3.321,3.202243,0.118757,3.58%,-0.057852
7,3.177,3.080849,0.096151,3.03%,-0.011949
8,2.719,2.963330,-0.244330,8.99%,-0.131136
9,3.023,2.849564,0.173436,5.74%,0.129281


In [384]:
gm_model.plot_res(xlabel="2015-2017季度",ylabel="案件数量")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 线性拟合


In [285]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [286]:
def polynomial_model(degree=1):
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    linear_regression = LinearRegression(normalize=True)
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    return pipeline

In [287]:
X=np.arange(12).reshape(-1,1)
Y=np.array(df_ym_num.values).reshape(-1,1)/1000

In [288]:
Y

array([[4.012],
       [3.761],
       [3.66 ],
       [3.532],
       [3.46 ],
       [3.629],
       [3.321],
       [3.177],
       [2.719],
       [3.023],
       [2.8  ],
       [2.358]])

In [289]:
degrees = [1, 3, 5, 10]
results = []
for d in degrees:
    model = polynomial_model(degree=d)
    model.fit(X, Y)
    train_score = model.score(X, Y)
    mse = mean_squared_error(Y, model.predict(X))
    results.append({"model": model, "degree": d, "score": train_score, "mse": mse,'RMSE':np.sqrt(mse)})
results

[{'model': Pipeline(steps=[('polynomial_features',
                   PolynomialFeatures(degree=1, include_bias=False)),
                  ('linear_regression', LinearRegression(normalize=True))]),
  'degree': 1,
  'score': 0.8962555793414528,
  'mse': 0.022496360916860938,
  'RMSE': 0.14998786923235138},
 {'model': Pipeline(steps=[('polynomial_features',
                   PolynomialFeatures(degree=3, include_bias=False)),
                  ('linear_regression', LinearRegression(normalize=True))]),
  'degree': 3,
  'score': 0.9184471768135963,
  'mse': 0.017684244921744915,
  'RMSE': 0.13298212256444442},
 {'model': Pipeline(steps=[('polynomial_features',
                   PolynomialFeatures(degree=5, include_bias=False)),
                  ('linear_regression', LinearRegression(normalize=True))]),
  'degree': 5,
  'score': 0.9396800629966126,
  'mse': 0.013080019770670167,
  'RMSE': 0.11436791407851316},
 {'model': Pipeline(steps=[('polynomial_features',
                   Polynomia

In [310]:
from matplotlib.figure import SubplotParams
plt.figure(figsize=(9, 6), dpi=200, subplotpars=SubplotParams(hspace=0.3))
all_x=np.arange(22).reshape(-1,1)
for i, r in enumerate(results):
    fig = plt.subplot(2, 2, i + 1)
    plt.xlim(0, 22)
    plt.title("LinearRegression degree={}".format(r["degree"]))
    plt.scatter(X, Y, s=5, c='b', alpha=0.5)
    plt.plot(all_x, r["model"].predict(all_x), 'r-')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [311]:
mse = mean_squared_error(res_df['原始值'][:12], res_df['预测值'][:12])
print("灰色预测",{"mse": mse,'RMSE':np.sqrt(mse)})

灰色预测 {'mse': 0.025310297904259395, 'RMSE': 0.15909210509720273}


## 幂函数拟合

In [312]:
from scipy.optimize import curve_fit

In [333]:
def fund(x, a, b):
    return -x**a + b
xdata=np.arange(12)
ydata=np.array(df_ym_num.values)/1000
popt, pcov = curve_fit(fund, xdata, ydata)
#popt数组中，三个值分别是待求参数a,b,c
y2 = [fund(i, popt[0],popt[1]) for i in xdata]
print(y2)
print(ydata)
print(len(y2))

[4.700944628373652, 3.7009446283736516, 3.5019459261666057, 3.367654374534397, 3.263346740479471, 3.176851924048842, 3.1023313443550826, 3.036489460403104, 2.977266626492938, 2.923281727390908, 2.873559453844978, 2.8273831520939607]
[4.012 3.761 3.66  3.532 3.46  3.629 3.321 3.177 2.719 3.023 2.8   2.358]
12


In [334]:
%matplotlib widget
plt.scatter(xdata, ydata, s=5, c='b', alpha=0.5)
plt.plot(xdata,y2,'r-')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [337]:
mse = mean_squared_error(ydata, y2)
print("幂函数预测",{"mse": mse,'RMSE':np.sqrt(mse)})

幂函数预测 {'mse': 0.09527403535238894, 'RMSE': 0.308664924072025}
